In [1]:
from selenium.webdriver import Chrome 
from bs4 import BeautifulSoup
import pymongo
import datetime
import time 
import pandas as pd
import re

In [2]:
mc = pymongo.MongoClient()

In [6]:
db = mc['chordify']

In [7]:
raw_html = db['raw_html']

In [68]:
raw_html.find().count()

920

In [278]:
len(list(raw_html.find()))

920

In [279]:
html_docs = list(raw_html.find())

In [280]:
doc = html_docs[579] 

In [281]:
html = doc['html']

In [282]:
soup = BeautifulSoup(html, 'html.parser')

In [283]:
song_body = soup.select_one('pre._1YgOS')
song_body

<pre class="_1YgOS" style='font-size: 13px; font-family: "Roboto Mono", monospace;'>Here is a tab on how to play Bob Dylan's ''Most Of The Time'' from his 1989 album ''Oh Mercy''.

Intro: <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">F</span> |<span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Bb</span> | <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C7</span> | <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">F</span>






             <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>
Most of the time
      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Fadd9</span>
I’m clear focused all around
             <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>
Most of the time
                <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Fadd9</span>
I can keep both feet on the ground
<span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Am</span>                  <span class="_3L0Da _3f5kI" style

In [284]:
lines = str(song_body).split('\n')
lines

['<pre class="_1YgOS" style=\'font-size: 13px; font-family: "Roboto Mono", monospace;\'>Here is a tab on how to play Bob Dylan\'s \'\'Most Of The Time\'\' from his 1989 album \'\'Oh Mercy\'\'.',
 '',
 'Intro: <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">F</span> |<span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Bb</span> | <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C7</span> | <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">F</span>',
 '',
 '',
 '',
 '',
 '',
 '',
 '             <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>',
 'Most of the time',
 '      <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Fadd9</span>',
 'I’m clear focused all around',
 '             <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">C</span>',
 'Most of the time',
 '                <span class="_3L0Da _3f5kI" style="color: rgb(0, 0, 0);">Fadd9</span>',
 'I can keep both feet on the ground',
 '<span class="_3L0Da _3f5kI" style="c

In [285]:
def strip_html(text):
    result = []
    in_tag = False
    for char in text:
        if char == '<':
            in_tag = True
        if not in_tag:
            result.append(char)
        if char == '>':
            in_tag = False
    return ''.join(result)

In [286]:
def parse_lines(lines):
    parsed_lines = []
    for line in lines: 
        if '_3L0Da' in line:
            parsed_lines.append({'chords': strip_html(line)})
        elif line and strip_html(line) == line:
            parsed_lines[-1]['words'] = line
    return parsed_lines 

In [287]:
def parse_lines_2(lines):
    parsed_lines = []
    for i, line in enumerate(lines): 
        if '_3L0Da' in line:
            parsed_lines.append({'chords': strip_html(line)})
        elif (('_3L0Da' in lines[i-1]) and (line == line) and (strip_html(line) == line)):
            parsed_lines[-1]['words'] = line
    return parsed_lines 

In [288]:
parsed_lines = parse_lines_2(lines)
parsed_lines

[{'chords': 'Intro: F |Bb | C7 | F', 'words': ''},
 {'chords': '             C', 'words': 'Most of the time'},
 {'chords': '      Fadd9', 'words': 'I’m clear focused all around'},
 {'chords': '             C', 'words': 'Most of the time'},
 {'chords': '                Fadd9',
  'words': 'I can keep both feet on the ground'},
 {'chords': 'Am                  G F ',
  'words': 'I can follow the path, I can read the signs'},
 {'chords': 'Am               G  F',
  'words': 'Stay right with it when the road unwinds'},
 {'chords': '                C                   Fadd9',
  'words': 'I can handle whatever I stumble upon'},
 {'chords': '              Am     G     F',
  'words': 'I don’t even notice she’s gone'},
 {'chords': '             C   F  C  F ', 'words': 'Most of the time'},
 {'chords': '             C', 'words': 'Most of the time'},
 {'chords': '    Fadd9', 'words': 'It’s well understood'},
 {'chords': '              C', 'words': 'Most of the time'},
 {'chords': '     Fadd9', 'word

In [289]:
type(parsed_lines[0])

dict

In [290]:
def get_chords(parsed_line):
    chord_idxs = []
    chords = []
    c_string = parsed_line['chords']
    for chord in re.finditer('\w+', c_string):
        chord_idxs.append(chord.start())
        chords.append(chord.group())
    chord_tups = list(zip(chord_idxs, chords))
    return chord_idxs, chords, chord_tups

def get_words(parsed_line):
    word_idxs = []
    words = []
    if 'words'in parsed_line: 
        w_string = parsed_line['words']
        for word in re.finditer('\w+', w_string):
            word_idxs.append(word.start())
            words.append(word.group())
    word_tups = list(zip(word_idxs, words))
    return word_idxs, words, word_tups

In [291]:
def merge_ch_word(parsed_line):
    word_tups = get_words(parsed_line)[2]
    chord_tups = get_chords(parsed_line)[2]
    word_list = get_words(parsed_line)[1]
    word_chord_idx_list = []
    for ch_tup in chord_tups: 
        for i, wd_tup in enumerate(word_tups):
            if wd_tup[0] > ch_tup[0]:
                word_chord_idx_list.append((i-1, ch_tup[1]))
                break 
    return (word_chord_idx_list, word_list)

def combine_ch_wd_list(merged_line_0, merged_line_1):
    ch_tups_0, wd_list_0 = merged_line_0
    ch_tups_1, wd_list_1 = merged_line_1
    new_tups = []
#     combo_lists = []
    for ch_tup_1 in ch_tups_1:
        new_tups.append( ( ((ch_tup_1)[0]+len(wd_list_0)), ch_tup_1[1] ) )
    combo_tups = ch_tups_0 + new_tups 
    combo_lists = wd_list_0 + wd_list_1
    return combo_tups, combo_lists 

In [292]:
def combo_all(parsed_lines):
    for i, parsed_line in enumerate(parsed_lines):
        if i == 0:
            merged_line = merge_ch_word(parsed_line)
        else: 
            merged_line_next = merge_ch_word(parsed_line)
            merged_line = combine_ch_wd_list(merged_line, merged_line_next)
    return merged_line 

In [293]:
merged_line = combo_all(parsed_lines)
merged_line

([(6, 'Fadd9'),
  (18, 'Fadd9'),
  (22, 'Am'),
  (26, 'G'),
  (26, 'F'),
  (32, 'Am'),
  (35, 'G'),
  (36, 'F'),
  (43, 'C'),
  (51, 'Am'),
  (52, 'G'),
  (64, 'Fadd9'),
  (72, 'Fadd9'),
  (79, 'Am'),
  (85, 'G'),
  (86, 'F'),
  (91, 'Am'),
  (96, 'G'),
  (97, 'F'),
  (104, 'C'),
  (113, 'Am'),
  (114, 'G'),
  (124, 'Fadd9'),
  (133, 'Fadd9'),
  (140, 'Am'),
  (145, 'G'),
  (146, 'F'),
  (151, 'Am'),
  (156, 'G'),
  (157, 'F'),
  (166, 'C'),
  (172, 'Am'),
  (175, 'G'),
  (184, 'Am'),
  (195, 'Am'),
  (204, 'C'),
  (209, 'E7'),
  (213, 'Am'),
  (219, 'E7'),
  (225, 'Am'),
  (235, 'Fadd9'),
  (243, 'Fadd9'),
  (249, 'Am'),
  (254, 'G'),
  (254, 'F'),
  (261, 'Am'),
  (264, 'F'),
  (272, 'C'),
  (282, 'Am')],
 ['Most',
  'of',
  'the',
  'time',
  'I',
  'm',
  'clear',
  'focused',
  'all',
  'around',
  'Most',
  'of',
  'the',
  'time',
  'I',
  'can',
  'keep',
  'both',
  'feet',
  'on',
  'the',
  'ground',
  'I',
  'can',
  'follow',
  'the',
  'path',
  'I',
  'can',
  'read',
  

In [294]:
len(merged_line[1])

293

In [273]:
get_words(parsed_lines[21])

([0, 4, 8, 12, 15, 21, 26, 29, 35, 40, 43],
 ['And', 'now', 'you', 're', 'gonna', 'have', 'to', 'get', 'used', 'to', 'it'],
 [(0, 'And'),
  (4, 'now'),
  (8, 'you'),
  (12, 're'),
  (15, 'gonna'),
  (21, 'have'),
  (26, 'to'),
  (29, 'get'),
  (35, 'used'),
  (40, 'to'),
  (43, 'it')])

In [272]:
parsed_lines

[{'chords': 'Fmaj7/C  x33210'},
 {'chords': 'Dm7      xx0211', 'words': ''},
 {'chords': 'C  Fmaj7/C   C   Fmaj7/C   C   Fmaj7/C   C    Fmaj7/C',
  'words': ''},
 {'chords': 'C                    Dm7',
  'words': 'Once upon a time you dressed so fine'},
 {'chords': '    Em                    F            G',
  'words': "You threw the bums a dime in your prime,   didn't you?"},
 {'chords': 'C                      Dm7',
  'words': '  People\'d call, say, "Beware doll, '},
 {'chords': '       Em                         F             G',
  'words': 'you\'re bound to fall" You thought they were all   kiddin\' you'},
 {'chords': 'F             G', 'words': '  You used to   laugh about'},
 {'chords': 'F                    G',
  'words': "  Everybody that was   hangin' out"},
 {'chords': 'F         Em   Dm7         C',
  'words': " Now you don't talk so loud"},
 {'chords': 'F         Em   Dm7          C',
  'words': " Now you don't seem so proud"},
 {'chords': '     Dm7                     F  